# INFO 2950 Final Project

In [1]:
# Load libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Load in the Stop and Frisk (2011) raw dataset:

In [2]:
SandF_2011_raw = pd.read_csv('2011.csv', encoding = 'latin1')


/Users/leajih-vieira/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (73,103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Because we are using two Stop and Frisk datasets from 2011 and 2019, we must match columns to each other and assign them identical column names for ease of analysis. We also must identify columns that are unable to be used since the data were only recorded in one dataset and not the other, etc. The following steps deal with this process.

Create a list of the column names that will be dropped from the SandF_2011 dataset titled drop_cols_for_2011. Drop the columns in this list that are unnecessary for our project:

In [3]:
drop_cols_for_2011 = ['pct', 'ser_num', 'inout', 'trhsloc', 'typeofid', 'sumoffen', 'compyear', 'comppct', 'offunif', 'officrid', 'adtlrept', 'radio', 'ac_rept', 'ac_inves', 'ac_proxm', 'ac_evasv', 'ac_assoc', 'ac_cgdir', 'ac_incid', 'ac_time', 'ac_stsnd', 'ac_other', 'repcmd', 'revcmd', 'offverb', 'offshld', 'forceuse', 'dob', 'ht_feet', 'ht_inch', 'weight', 'haircolr', 'eyecolor', 'build', 'othfeatr', 'addrtyp', 'rescode', 'premtype', 'premname', 'addrnum', 'stname', 'stinter', 'crossst', 'aptnum', 'state', 'zip', 'addrpct', 'sector', 'beat', 'post', 'xcoord', 'ycoord', 'dettypcm', 'linecm', 'detailcm']

SandF_2011 = SandF_2011_raw.drop(labels = drop_cols_for_2011, axis = 1)


Rename columns so that they have more easily understandable names:

In [4]:
SandF_2011 = SandF_2011.rename(columns={"datestop": "date", "timestop": "time", "perobs": "obs_duration", "crimsusp": "crime_sus", "perstop": "stop_duration", "explnstp": "off_explain", "othpers": "other_stop", "contrabn": "contraband", "knifcuti": "knife", "othrweap": "other_weapon", "city": "boro"})


The Stop and Frisk (2011) dataset has data recorded on if weapons were found on the supsect, and if so, what kind of weapon it was. There are multiple differences between the 2011 and 2019 dataset on the weapons categories. To solve this difference between the datasets, we decided to reorganize the weapons columns into three categories: firearm, knife, and other weapon. The following cell reformats the weapons categories and renames them. 

In [5]:
sus_firearm_str = SandF_2011.pistol + SandF_2011.riflshot + SandF_2011.asltweap + SandF_2011.machgun

sus_firearm = sus_firearm_str.str.contains(pat = 'Y')

# Add the firearms column to the SandF_2011 DataFrame.
SandF_2011['firearm'] = sus_firearm

# Drop the unnecessary firearms columns.
drop_cols_firearm = ['pistol', 'riflshot', 'asltweap', 'machgun']

SandF_2011 = SandF_2011.drop(labels = drop_cols_firearm, axis = 1)


When an officer uses physical force on a suspect, there are many categories that the physical force can be classified into: hands, suspect against wall, suspect on ground, weapon drawn, weapon pointed, baton, handcuffs, pepper spray, and other. We are mainly interested in whether or not a weapon was drawn, a weapon was pointed, and if the suspect was handcuffed. In order to condense the physical force data, we made a general physical force column that records whether or not any physical force was used during the encounter, but will be keeping the separate weapon drawn, weapon pointed, and handcuffs columns. These edits are done in the cell below:

In [7]:
phys_force_str = SandF_2011.pf_hands + SandF_2011.pf_wall + SandF_2011.pf_grnd + SandF_2011.pf_drwep + SandF_2011.pf_ptwep + SandF_2011.pf_baton + SandF_2011.pf_hcuff + SandF_2011.pf_pepsp + SandF_2011.pf_other

phys_force = phys_force_str.str.contains(pat = 'Y')

# Add the physical force column to the SandF_2011 DataFrame.
SandF_2011['phys_force'] = phys_force

# Drop the unnecessary physical force columns.
drop_cols_phys_force = ['pf_hands', 'pf_wall', 'pf_grnd', 'pf_baton', 'pf_pepsp', 'pf_other']

SandF_2011 = SandF_2011.drop(labels = drop_cols_phys_force, axis = 1)

SandF_2011.head()


,year,date,time,recstat,obs_duration,crime_sus,stop_duration,off_explain,other_stop,arstmade,...,sb_admis,sb_other,rf_furt,rf_bulg,sex,race,age,boro,firearm,phys_force
0,2011,1012011,0,A,1,BURGLARY,6,Y,N,N,...,N,N,Y,N,M,A,21,QUEENS,False,True
1,2011,1012011,5,A,1,FEL,2,Y,N,N,...,N,N,Y,N,M,B,15,QUEENS,False,False
2,2011,1012011,7,1,1,CPW,2,Y,Y,N,...,N,N,Y,N,M,B,17,QUEENS,False,True
3,2011,1012011,7,1,1,CPW,2,Y,Y,N,...,N,N,N,N,M,B,17,QUEENS,False,True
4,2011,1012011,7,1,1,CPW,2,Y,Y,N,...,N,N,N,N,M,B,20,QUEENS,False,True


In [8]:
SandF_2011.to_csv(path_or_buf = 'SandF_2011_size_test')